In [3]:
import ast

In [23]:
code1 = """
# pets_factory.py

class Dog:
    def speak(self):
        return "Woof!"

class Cat:
    def speak(self):
        return "Meow!"

class Bird:
    def speak(self):
        return "Chirp!"

# Function to create objects dynamically
def create_pet(pet_type):
    pets = {
        "dog": Dog,
        "cat": Cat,
        "bird": Bird
    }
    cls = pets.get(pet_type.lower())
    if cls:
        return cls()
    else:
        raise ValueError(f"Unknown pet type: {pet_type}")


"""

code2 = """
# main.py

from pets_factory import create_pet

def main():
    pet_names = ["dog", "cat", "bird"]

    for name in pet_names:
        pet = create_pet(name)
        print(f"A {name} says: {pet.speak()}")

if __name__ == "__main__":
    main()

"""

In [27]:
tree1 = ast.parse(code1, filename="pets_factory.py")
tree2 = ast.parse(code2, filename="main.py")

In [28]:
def get_defs(tree):
    classes = []
    functions = []
    for node in ast.walk(tree):
        if isinstance(node, ast.ClassDef):
            classes.append(node.name)
        elif isinstance(node, ast.FunctionDef):
            functions.append(node.name)
    return classes, functions

classes1, funcs1 = get_defs(tree1)
classes2, funcs2 = get_defs(tree2)

print("File1 Classes:", classes1)
print("File1 Functions:", funcs1)
print("File2 Classes:", classes2)
print("File2 Functions:", funcs2)


File1 Classes: ['Dog', 'Cat', 'Bird']
File1 Functions: ['create_pet', 'speak', 'speak', 'speak']
File2 Classes: []
File2 Functions: ['main']


In [30]:
def get_func_calls(tree):
    calls = []
    for node in ast.walk(tree):
        if isinstance(node, ast.Call):
            if isinstance(node.func, ast.Name):
                calls.append(node.func.id)
            elif isinstance(node.func, ast.Attribute):
                calls.append(node.func.attr)
    return calls

calls2 = get_func_calls(tree2)
print("Function calls in file2:", calls2)


Function calls in file2: ['main', 'create_pet', 'print', 'speak']


In [31]:
def get_returned_classes(tree):
    returned = []
    for node in ast.walk(tree):
        if isinstance(node, ast.Return):
            if isinstance(node.value, ast.Call):
                if isinstance(node.value.func, ast.Name):
                    returned.append(node.value.func.id)
    return returned

returned_classes = get_returned_classes(tree1)
print("Classes returned dynamically:", returned_classes)


Classes returned dynamically: ['cls']
